## Datastrukturer

Pandas bygger på numpy arrays og arver mye funksjonalitet derfra. Det er to sentrale objekt: Series og DataFrame. DataFrame kan betraktes som en tabell og Series som en kolonne fra en tabell.

I tillegg til den implisitte numeriske indexen til arrays har de også et eksplisitt indexobjekt. I Series mapper verdi av index til enkeltverdi. I DataFrame mapper det til et Series-objekt. 

Koblingen mellom indeks og verdi gjør at vi kan kombinere data fra ulike tabeller, håndtere *missing values* og generelt ikke er avhengig av å ha helt konforme data (samme størrelse, samme rekkefølge) for å gjøre regneoperasjoner ...

### Series

Series er crossover mellom array og dict. Består av to arrays: én index og én med homogene data. Vanlige ndarrays har implisitt index med posisjon, men her er det eksplisitt og kan ha ulike labels. Kan også tenke på Series som en dict som mapper index-verdier til verdi i array. Følgelig er det en del metoder/syntax som tilsvarer dict. Kan få ut hver av arraysene med:
1. a.values (ndarray)
2. a.index (index-objekt, finnes litt ulike typer, har diverse metoder, noe mengdegreier)

### DataFrame

DataFrame er tabulær datastruktur med både rekke-index og kolonne-index. I utgangspunktet er det ganske symmetrisk, men av konvensjon angir rekke-index obseravjon og kolonne-index er variabel (dimensjon/egenskap) ved observasjonen. Kan betrakte det som en dict som mapper label til Series, der labels er enten fra rekke- eller kolonneindex.

### Index

Det er en fullverdi datastruktur/objekt i seg selv. Har en del metoder og sånn. Vet ikke om jeg må jobbe så mye direkte med indexen; kan gjøre operasjon på dataframe og så håndterer de index internt.


Merk at når vi setter en ny kolonne som indeks mister vi den eksisterende index-kolonnnen. Hvis vi vil beholde må vi lagre kopi av kolonne og deretter putte den inn i nye dataframe. Kan få tak i kolonnen ved å bruke df.index() og deretter dytte den inn i df ved å assigne kolonnen med df["navn"] = df.index()


#### Indeksing

Kan legge til verdier på subset av dataframe

```python
df.loc['a'] = arr
df.loc['a'] = df_sub # fungerer ikke selvom matchene index, vet ikke hvorfor ...
```

#### kolonner

Kan merke at kolonnene egentlig bare er index og at det i utgangspuntket er ganske symmetrisk.

Vil ha deskriptive kolonnenavn. For å endre bruker vi dict med mapping
```python
mapper = {'gammel':'ny'}
df = df.rename(columns=mapper)
```
Kan bruke vanlig list comprehension til å finne subset av kolonner som oppfyller kriterie,
```python
cols_subset = [col for col in df.columns if 'string' in col]
```

#### Multi index

Index med flere level. Ytterste er level=0, og så teller vi oppover 

##### Konstruere multiindex

Kan eksplisitt konstruere multiindex objekt på flere måter

```python
arrays = [np.array(["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"]),
          np.array(["one", "two", "one", "two", "one", "two", "one", "two"])]

# Tre ekvivalente indekser:
pd.MultiIndex.from_tuples(zip(arrays[0],arrays[1])) # trenger list of tuples
pd.MultiIndex.from_arrays(arrays) # trenger list of arrays
pd.MultiIndex.from_frame(pd.DataFrame(arrays).T) # organisere arrays i dataframe før vi lage index

pd.MultiIndex.from_product(iterables) # alle unike kombinasjoner av verdier i ulike arrays
```

Kan også konstruere MultiIndex i constructor for DataFrame, må da være list of arrays
```python
df = pd.DataFrame(index=arrays, data=np.random.randn(len(index)), columns=['tall'])
df.set_index('tall', append=True) # legge til kolonne på eksisterende index

# Hvis vi ikke har eksisterne index kan vi lage nye multiindex med:
df.set_index(["kol1","kol2"])
```

Hvis vi har lyst til å legge ny array (eller eventuelt konstant verdi) som øvereste level i multiindex bør vi transformere index til dataframe så den  blir enklere å manipulere
```python
temp = df.index.to_frame()
temp.insert(0, 'name', value)
df.index = pd.MultiIndex.from_frame(temp)
```

##### Slice multiindex

Subsetting av observasjoner avhengig av verdier i multiindex kan bli ganske komplisert.
```python

df.loc['a'] # alle rekker der index i level 0 == 'a'
df.loc[('a', 'b')] # alle rekker der level 0 == 'a' og level 1 == 'b'
```

Må se mer på dette senere

#### Sortering

Kan sortere entent etter index eller verdi langs gitt kolonne,
```python
df.sort_index()
df.sort_values(['index','kol'])
```

## Lage dataframes

Vi kan enten initialisere Series og DataFrame fra filer (lokalt på disk eller i sky) eller fra andre objekter i Python.

### Fra fil

De to vanlige lagringsformatene for tabulære data er .csv og .json

#### Fra csv

Tekstfil der rader er separert på ulike linjer `\n` og verdier er separerert med delimiter (gjerne `,` eller `;`). Siden det er en tekstfil kan vi åpne den i notepad for å inspisere strukturen.

```python
pd.read_csv(filename,
            sep=',', # kan bruke andre seps, tror tab er '\t'
            delimiter=None,
            header='infer', # må spesifisere header=0 for å endre navn..
            names=None, # hvis vi vil spesisifisere navn på kolonne
            index_col=None, # hvis første kol skal være index bruker vi =0
            usecols=None, # hvis vi vil spesifisere subset av kolonner
            parse_dates=True, # finner kolonne med date, gjør den til index, får den i datetime.
            skiprows=2, # hoppe over rader
            dtype = {'name':'dtype'}, # tilsvarer å calle df.name.astype('dtype') ex-post
            converters = {'name':func}, # tilsvarer df.name.apply(func) ex-post
            ...)
```

#### Fra json

Det andre vanlige lagringsformatet for tabulære data er json (javascript object notation). Det korresponderer i stor grad med dictionary i Python. Må spesifisere `orient`
1. split
2. records
3. columns

### Fra datastrukturer

Vi kan også initialisere fra andre objekter i Python som representerer data (dictionary og arrays).
```Python
pd.Series(data=x,index=y). 
pd.Series(dict(zip(y,x))
```          

### Lagre dataframe

Kan lagre dataframes som består av tall og strenger som tekst i csv format med `to_csv`

Hvis det inneholder andre objekter, så må vi lagre til `to_pickle`. Generelt vil pickle lagre fullstendig informasjon om dataframe; alt fra datatyper, form av indeks, ... alt. Når vi lagrer til csv så er det bare en lang string og vi må tolke denne stringen når vi laster dataframe, så da starter vi gjerne fra scratch. Pickle er bedre.

## Beskrive data

Før jeg kan gjøre noe analyse må jeg først undersøke tabellen med tall jeg har fått utdelt. Må få oversikt over:
1. Hvilke egensakper (kolonner) og hva de måler
2. Hva slags måleenhet de er i og konvertere til riktig datatype. I algebraen under the hood er alt tall, men representer som factor for å få bedre representasjon og utnytte funksjonalitet i plotte-libraries og lignende. Vil unngå å jobbe eksplisitt med dummies før det er nødvendig.
3. Få noe oversikt over univariate fordelinger og eventuelt parvis bivariate korrelasjoner. Se at tall er rimelig, blir kjent med data, oppdage numerisk kodete missing values, se etter feilkodinger
4. Få oversikt over manglende verdier, vurdere strategi for imputation og eventuelt bias dersom vi ser på delutvalg med fullstendig obervasjon.

```python
df.describe() # output avhenger av datatype
df.info() # datatype og observere om det er missing values
df.ndtype() # finne datatypene
df['col'].value_count(normalize=False) # antallet observasjoner med gitte verdier langs den dimensjonen, eller andel
df['col'].unique() # liste av unike verdier
```

### Datatyper

Hver kolonne har en består av verdier med homogen datatype. Hvordan verdiene blir representert og hvilke funksjonaliterer som er gyldige avhenger av datatype, og det er derfor viktig å ha oversikt over dette. Pandas forsøker å tolke datatype hvis det ikke er eksplisitt spesifisert. Hvis ikke homogen representasjon vil den forsøke å konvertere verdier (f.eks. float og int blir til float).

Vil håndtere spesifikasjon av datatyper gjennom `read_csv` for å separere det fra analysen. Kan være nødvendig med custom funksjon som 
```python
def convert_currency(val):
    return float(val.replace(',','').replace('$',''))
df['col'] = df['col'].apply(convert_currency)
```

#### Numerisk

Kan enten være `int` (heltall) eller `float`. Kan spesifisere hvor mange bits vi vil sette av til å representere hver av tallene. Høyere bits betyr mer lagringsplass, men kan representere høyere tallverdier.

Numeriske kolonner med manglende verdier blir konvertert til float. Kan ha lyst til å representere dette som ints...

Har hjelpefunksjon for å konvertere til numerisk når .astype() ikke klarer å tolke
```python
pd.to_numeric(col, errors='coerce') # nans for verdier som ikke kan tolkes som numerisk
```

#### Kategorisk

Tror det korresponderer med `factor` i `R`. Tror vi trenger representasjonen i statistisk analyse.. Liste av teksverdier.
```python
pd.Categorical(df.col, ordered=True, categories = [..]) # spesifisere rekkefølge hvis ordinal

# Konvertere object til category
cat_cols = [col for col in df.columns if df[col].dtype == 'object']
df[cat_cols] = df[cat_cols].astype('category')
```

#### Objekt

Strings eller mixed type .. Tror hvis mixed så har ikke verdiene homogen datatype

##### String

Hvis kolonne er string kan jeg bruke df.col.str for å få tilgang til metoder. Dette er ofte bedre å raskere enn å gjøre en eksplisitt loop og bruke string metode på innholdet i hver celle. Har de fleste (alle?) metodene som native python har på strings. Har i tillegg noen egne funskjonaliteter knyttet til pandas, feks:
```python
# lage dummmies der hver observasjon kan inneholde flere verdier i form av string
df['col'].str.get_dummies(sep='|') 

# Hvis jeg vil bruke flere str operasjoner i chain må jeg bruke .str etter hver, eks
data['Min_Salary']=data['Min_Salary'].str.strip(' ').str.lstrip('$').str.rstrip('K').fillna(0).astype('int')
```


#### Dato og tid

Tidspunkt har mange egenskaper. De har rangering der større verdier er senere enn små verdier, men det er langt fra tilstrekkelig å representere denne ordinale rangeringen med tall. Det finnes mange ulike målenheter på avstand i tid (dager, sekunder, millisekunder, mm.) og det er ikke alltid trivielt å konvertere mellom disse, samt at det avhenger av tidssone. Dessuten er det mye informasjon assosiert med gitte tidspunkt (hvilken uke eller dag det er, mm).

Dersom vi gir informasjon om tidspunkt en riktig representasjon kan pandas gjøre mye arbeid for oss. Det gir oss tilgang til informasjon og det kan tolke hva vi mener slik at det blir enkelt å filtrere observasjoner ut fra årstall, ukedag eller lignende. Det kan også tolke avstand mellom tidspunkt i ulike måleenheter. For å oppnå dette må vi bruke classer som er *time aware*.

Av uvisse grunner har det mer intuitiv funksjonalitet når datetime array er indeks i stedet for kolonne i dataframe... Gir vel uansett stort sett mening å ha det som index.

Har scalar-class og array-class (eks: timestamp og datetimeindex)

##### Timestamp

Representerer et punkt i tid. Ganske analog til datetime i base python og sql. Har hjelpefunksjon for å konvertere string-representasjon til timestamp.
```python
pd.to_datetime(array, # prøver å parse innhold i array, men vi må ofte i praksis hjelpe litt til med flere argument
               format, # spesifisere format, litt usikker på hvordan... eks '%Y%m%d' for '2010/11/12'
               day_first, # alternativt kan vi gi litt tips
               infer_datatime_format=True, # prøver å gjette fra første ikke-nan. Kan være ambiguøst, så forsiktig ..
               )
```

Kan konstruere index som består av datetime med 
```python
date_index = pd.date_range(start, # string som kan bli parset som dato..
                           period, # int med antall perioder
                           freq, # avstand mellom perioder, 'D' for daily, 'H' hourly, ..
                           end, # kan alternativt spesifisere sluttdato
                           tz # kan spesifisere timezone. Kan deretter konverte med .tz_convert('time zone')
                           )
```

Bruker .dt til å få interface til egenskaper for datetime objekt

##### Timedelta

Differanse i tid mellom to timestamps. Kan være resultat av artimetikk på timestamps..
```python
pd.Timedelta('3 days')
pd.to_timedelta('3 days')
```

##### Period

Representerer intervallet av tidspunkt mellom to tidspunkt.

## Missing values

Vi vil representere manglende verdier. Default er `pd.NaN` og og `np.NaT`. `np.NaN` er en float, slik at numeriske kolonner med missing values blir recast til floats. Det er lagt til et alternativ objekt `pd.NA` som representere atomisk verdi og tar sikte på å være uavhengig av datatype. Kan bruke i nullable integer,
```python
df['col']=df['col'].astype('Int64') # stor bokstav
```
Missing values propapegerer når vi gjør operasjoner på dataframe. Operasjon er elementvis og alt som bruker NaN resultererr i NaN. Aggregeringsfunksjoner pleier å ignorere NaN

### Filtrering

Første jeg må gjøre er å sjekke antallet missing values i ulike kolonner. Husk at missing values kan være kodet på ulike måter i ulike datasett. En mulighet er å bruke df['col'].unique() til å se på verdier og se om noen ikke gir mening. Deretter vil jeg omkode de til NaN, f.eks. ved df.replace(value,np.nan). Når jeg har riktig koding kan jeg konstruere boolean mask for å identfisere
```python
df.isna() # gir boolean mask over hele dataframe
df.isna().any() # angir om hvorvidt det er nans i hver av kolonner
df.isna().sum() # antallet nans i hver kolonne

# 
df.dropna(axis, # drop rad (0) eller kolonne (1) avhengig om missing i celle
          how, # {'any', 'all'} # drop hvis noen missing eller alle missing langst angitt akse
          ..)
```

### Imputation

En alternativ fremgangsmåte er å fylle inn verdier. Algoritmer i sklearn kan ikke håndtere nans og det er synd å kaste bort data bare fordi noen observasjoner er litt mangefulle. Finnes ulike strategier for dette som jeg kan se på senere
1. Bruke gjennomsnitt i kolonnen
2. Predikere verdi ut fra andre dimensjoner vi observere
3. Bruke verdi fra naboer

## Split-apply-combine

Vi har et stort datasett. Det er mange variabler og mange kolonner. Hvordan skal vi få ut noen innsikter fra all denne informasjonen?

Kan vært veldig nyttig å dele det inn i mindre deler etter felles verdi langs en (eller flere) kolonne(r). Kan tenke at det er observasjoner som "hører sammen". Deretter annvender vi noen aggregeringsfunksjoner som gir oss slags sammendragsmål for tallverdiene i hver del. Deretter kan vi kombinere det sammen i nytt datasett som mapper nøkkel (unike(e) verdi(er) av identifiserende kolonne(r)) til verdi av aggregeringsfunksjon.

### groupby-objekt

```python
df.groupby(by, # kolonnenavn eller liste av kolonnenavn som vi grupperer etter. Eller pd.Grouper..
           level, # hvis vi vil bruke kolonner fra multiindex
           as_index, # brukt som index i output.. kan sette false for å sikre at dataframe
           observed # kan brukes hvis grupperingskolonne er `Categoricals` .. tror ikke så relevant
           )
```
Det er en måte å konstruere separate dataframes filtrert på verdi av kolonne vi grupper etter. Kan få tak i de interne dataframene med
```python
gb = df.groupby('col')
for idx, df in gb:
    print(idx) # verdi av col
    print(df) # tilsvarer df[df['col'==val]] for hver unik val i 'col'
```    
I praksis vil vi ikke jobbe direkte med tabellene internt i groupby-objektet, men i stedet bruke metoder.

### Aggregering

Kan bruke hvis vi vil kjøre ulike aggegreringsfunksjon på ulike kolonner
```python
df_out = (df.groupby("key").
              agg({"col1":'mean', 'col2':'count'}).
              rename(columns={'col1':'avg_col1', 'col2':'num_col2'}))
```
Det ser veldig stygt ut, så vi bruker i stedet `NamedAgg` objekt,
```python
df_out = df.groupby("key").agg(
    avg_col1 = pd.NamedAgg('col1', 'mean'),
    num_col2 = pd.NamedAgg('col2', 'count'))
```

### Multi-level groupby

```python
df_out = df.groupby(['first_column', 'second_column'])['some_column'].mean()
```
Merk at df_out bare vil ha index for de kombinasjoner av verdier i kolonnene vi grupperer på som vi faktisk observerer i data. I mange tilfeller kan det være greit å ha alle kombinasjonene og padde med NaNs for de vi ikke observerer. Litt usikker på hvordan jeg skal gjøre dette... må resette/konstruere denne indeksen på en måte.

### Grouper

Hvis vi har lyst til å gruppere etter verdier på en kolonne, men den har for fin inndeling (for mange unike verdier). Da kan vi bruke `pd.Grouper` til å lage en grovere inndeling ved å lag bins med flere verdier og gruppere sammen alle radene som korresponderer med samme bins. Brukes i praksis på kolonner med tidsdimensjon.

```python
# gruppere observasjoner med timedelta innefor 5 minutters intervall
duration_agg = df.groupby(pd.Grouper(key='duration',freq='5Min'))['started_at'].agg('count')

# Kan kombinere med annen key for å få gjennomsnitt av størrelse som varierer over tid innad i hver kategori..
trips.groupby(['name', (pd.Grouper(key='start_ts',freq='D'))]).car_id.nunique().groupby('name').mean()
```

## Reshape

Kan tenke at kombinasjonen av en index-verdi og kolonne-verdi til sammen utgjør en tuple som identifiserer en celle med verdi i tabellen. Vi kan flytte ting mellom index og kolonne og likevel identifisere de samme cellene med verdier, men tabellen vil se ganske annerledes ut.

For analyser og visualiseringer vil vi ha data på *tidy*-format der hver rad er en observasjon og hver kolonne en egenskap ved denne observasjonen.

Tror ikke det er noen entydig definisjon på *wide* og *long*, men i long flytter vi identifikatorer ned til index og i wide så går det opp i liste av kolonnenavn...

### High-level

Har to high-level funksjoner. Pivot som gjør *wide* og melt som gjøre *long* ...

#### Pivot

```python
df.pivot(index, # hver unik verdi i kolonnen utgjør én rekke i ny df
         columns, # hver unik verdi utgjør navn på én kolonne i ny df
         values=, # hvilke kolonner som fyller verdiene i celler
        )
```

Eksempel
```python
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df.pivot(columns='foo', # kolonner er ['one', 'two']
         index='bar', # index er ['A', 'B', 'C']
         values='baz') # fyller inn verdi fra baz som korresponderer med verdiene av (foo, bar)
```

##### pivot_table

Generalisering som kan håndtere duplikat av (index, col)... Aggregering ?

#### Melt

*Unpivot* fra wide til long
```python
df.melt(id_vars, # kombinasjon av kolonne som unik identifiserer obs
        value_vars, # bruker alle som ikke er i id_vars som default.. kan spesifisere for å droppe resten..
        var_name, # navn på kolonnen med verdier fra opprinnelige kolonner
        value_name) # navn på kolonnen med verdi som korresponedere med opprinelig kolonner
```

Bruker hvis hver rekke korresponderer med mer enn én observasjon. Hva om hver rekke korresponderer med en dato og har observasjon for flere enheter på hver dato. For å gjøre tidy vil vi én observasjon per enhet per dato.
```python
df = pd.DataFrame({'tid': {0: 'a', 1: 'b', 2: 'c'},
                   'obs_A': {0: 1, 1: 3, 2: 5},
                   'obs_B': {0: 2, 1: 4, 2: 6}})
df.melt(id_vars='tid',value_vars=['obs_A','obs_B'])
```

### Low-level

Tror jeg kan gjenskape atferd i pivot og melt med `stack`, `unstack`, `set_index` og `reset_index`. Trenger da ikke å bruke high-level convinence funksjoner, og trenger derfor ikke huske atferden deres..

#### Stack

Legger til nytt nivå innerst i multiindex og sender kolonnenavnene dit. Alternativ måte å identifisere hvilken identifikator (row/column) som korresponderer med hvilken celle. Returnerer Series siden vi ikke lenger har kolonner (såfremt det ikke var multiindex i kolonnene)
```python
df = pd.DataFrame([[0, 1], [2, 3]],index=['cat', 'dog'], columns=['weight', 'height'])
df.stack()
```

#### Unstack

Den inverse operasjonen der vi tar de unike verdiene fra innerste nivå av multiindex og bruker de til å konstruere kolonneindex. Returnerer DataFrame. Trenger ikke være innerste, men det er default..

## Transformasjoner

I praksis er det mange innebygde funksjoner som kjører elementvis by default, så dette er ikke så veldig vesentlig tror jeg

### Map

Vil gjøre funksjon elementvis på rekke. Gir ut array med samme størrelse der output i rekken avhenger av input i rekken. 

Kan ta dict som argument hvis vi vil omkode verdier i henhold til tabell
```python
table = {'old_val0':'new_val0', 'old_val1':'new_val1'}
df['col'] = df['col'].map(table)
```
Kan også bruke lambda for å lage custom funksjoner
```python
df['col'] = df['col'].map(lambda x: 2*x) # x er verdi i rekken av array
```

### Apply

Metode til Series og DataFrame. På series caller den funksjon elementvis på innhold. Bruker til å finne andeler av gruppe etter groupby,
```python
antall = df.groupby('key')['col'].sum() # samlet antall for ulike verdier av 'col' for hver verdi av key
andel = antall.groupby(level=0).apply(lambda x: round(x/x.sum(), 2)) # andel av samlet antall for gitt key
```
der hvert argument i apply er series som korresponderer med ulike verdier av key. Det er litt lite ryddig at jeg må kjøre groupby to ganger siden jeg binder til midlertidig variabel og deretter concat/merger. Vet ikke hva som er best practice.

#### Applymap

hmm

### pipe

Tingen er at .apply() anvender funksjon på hvert element av iterable, men har ikke helt bilde av helheten.

Hvis jeg i stedet bruke .pipe() på groupby så vet den hvor mange grupper det er og sånn.

hmmm

### Where

Kan lage nye kolonne fra ifelse conditional med

```python
df['new_col'] = np.where(df['old_col']=='value', 'A', 'B') # kan bruke til å omkode fra string til boolean
```

## Kombinere dataframes med joins

Se SQL for beskrivelse av ulike joins. Tror jeg i praksis vil bruke concat og og merge i stedet for join i pandas.

### concat

Hvis har samme kolonner eller samme index i to dataframes så er det enklest å bare slenge de sammen med
```python
pd.concat([liste av dfs],
          axis=0)
```


### merge

Bruker verdi i felles kolonner til å merge verdier. Tenk at vi har observasjon om hvilke land individer kommer fra. På bakgrunn av dette vil vi koble på mer informasjon om landet på hvert individ. 
```python
left_df.merge(right_df,
              on='key', # hvis kolonnene vi joiner på har samme navn
              left_on, right_on, # hvis de har ulike navn
              left_index, right_index, # sette True dersom vi bruker index i stedet for kolonnenavn
              suffixed = ('_x', '_y') # håndtere overlappende kolonnenavn
              ) 
```

### join

Tror jeg bruker merge i stedet ..

### Legge til series (rad og kolonne)

Hvis vi bruker df.append() så lages nytt objekt. Må sende en series eller dataframe som argument. 

```python
df = dt.append(pd.Series({dict: keys:values})) # keys korresponderer med kolonnenavn
```
       


## Filtrering

Avgrense til å betrakte delmengde av rader og rekker

### Subset av kolonner med reindex

For å velge delmengde av kolonner bruker vi .reindex()
``` python
df = df[col_lists] # stygt!
df = (df.reindex(columns=col_list).
      ... # andre metoder, chaine uten å binde til midlertidige dfs hele tiden. vakkert!
     )

```

### Boolean mask

Kan bruke bools til å indexe. Gir ut array som "bli lagt oppå dataframen". Får ut de verdiene som korresponderer med True.     

```python
df_new = df[df['col'] == verdi]
df_new = df[(df["col1"] > 0) & (df["col2"] == 0)]
```

Ikke ideelt at jeg må binde til ny variabel siden det ryddigere å chaine method calls (som i dplyr), skal derfor se på andre måter å anvende boolean mask. Tror jeg vil bruke query i stedet

### Query

Eneste argumenter er en string som blir parset av `pd.eval`. Ser litt u-pythonisk ut, men beste alternativ for å filtrere i dplyr-ish workflow
```python
df = (df.query("colname > value").
      query("colname1 > colname2"). # sammenligne verdier i ulike kolonner
      query("`col name1`==value"). # backtick for å håndtere kolonnenavn med whitespace
      query("colname == ['a', 'b']"). # sjekke om verdi er element i liste
      query("colname == @some_list"). # bruker @ til å referere til objekt ekstern objekt i namespace 
      query("colname1 > value & colname2 < value"). # kan kombinere predikat på vanlig måte
     )
```

## Diverse data cleaning 

### Eksempler

```python
# Omdefinere verdier i kolonne i henhold til tabell/dict
table = {'Amer-Indian-Eskimo':'Native','Asian-Pac-Islander':'Pacific'}
df['race'] = (df['race'].
    str.strip().
    astype('category').
    cat.rename_categories(table))
```

```python 
# Vil ha kolonnenavn i camel_case i stedet for SnakeCase
def camel_to_snake(s):
    return ''.join('_'+ch.to_lower() if ch.isupper() else ch for ch in s)
df.columns = [camel_to_snake(column) for column in df.columns]

```

```python
# fjerne alle symboler i string som ikke er numerisk
df['name'] = df['name'].map(lambda x: ''.join([ch for ch in x if ch.isdigit()]))
```

## Annet

### pd.cut()

Når vi vil dele kontinuerlige data inn i bins. Konstruere 'factor' med levels, kan deretter lage dummies

```python
pd.cut(array,
       bins, # Antall bins (lager slik at uniform antall i hver) eller konstruere med lik avstand mellom
       retbins=True, # spesifisere om returnere cutoffs til bins
       labels # kan bruke np.range(len(bins)) hvis kun interessert i relativ plassering
       )
```

bins kan ta ulike form:
1. int, spesifiserer antall bins, partisjonerer (min(x),max(x)) inn i like store intervall
2. sekvens av tall (a,b,c,..), lager intervall (a,b],(b,c] hvis right=True. Gir NaN hvis ikke i angitte intervall.
3. IntervalIndex, konsturert fra array av Interval


### Nyttige metoder

```python
s.pct_change() # hver observasjon fra i=1 -> n er erstattet med df[i]/df[i-1]-1. Gang med 100 for %
s.nlargest() # gir series med index og verdi til n største verdiene i en series
s.idxmax() # gir første index til høyeste verdi langs gitt index.. hvis axis=1 så får vi kolonnen med høyest verdi 
```

## Annet

Det er ganske greit å rekonstruere en kategorisk variabel fra dummies (f.eks for plotting purposes eller hvis jeg vil kjøre groupby på det). Kategorisk er egentlig alltid en bedre representasjon før vi kjører det inn i algoritme, så synes egentlig ikke jeg bør se noen fuckings dummies okay det dritet der bør det være mulig å skjule.

Kan bruke list comprehension til å finne subset kolonner med gitte egenskap
```python
subset_cols = [col for col in df.columns if col.startswidth('prefix')]
subset_df = df[subset_cols]

from pandas.api.types import is_numeric_dtype
num_cols = [col for col in df.columns if is_numeric_dtype(df[col])]
```

## Dask

Kan bruke `dask.dataframe` når vi har datasett som er for stort til å laste hele inn i minnet samtidig. Har api som er veldig analog til pandas dataframe
```python
import dask.dataframe as dd
df = dd.read_csv('filename.csv')
```